# November 2025 Expense Analysis

In [22]:
from pathlib import Path

import pandas as pd

DATA_PATH = Path("data/Expenses-2025.11.csv")
EXPECTED_COLUMNS = [
    "Posted Date",
    "Payee",
    "Amount",
    "ABS Amount",
    "Source",
    "Bas-Lux",
    "Category",
]

df = pd.read_csv(
    DATA_PATH,
    encoding="utf-8-sig",
    thousands=",",
)
df["Posted Date"] = pd.to_datetime(df["Posted Date"].str.strip(), format="%m/%d/%y")
df.columns = df.columns.str.strip()

missing_columns = sorted(set(EXPECTED_COLUMNS) - set(df.columns))
unexpected_columns = sorted(set(df.columns) - set(EXPECTED_COLUMNS))

if missing_columns or unexpected_columns:
    raise ValueError(
        "Column mismatch detected.\n"
        f"Missing: {missing_columns or 'None'}\n"
        f"Unexpected: {unexpected_columns or 'None'}"
    )

expenses_df = df[~df["Category"]
                 .isin([
                     "Transfer", 
                     "Credit Card Payment", 
                     "0", 
                     "Income, Investment"
                     ])].copy()

In [23]:
import matplotlib.pyplot as plt
from IPython.display import display

monthly_category = (
    expenses_df.assign(month=df["Posted Date"].dt.to_period("M"))
    .groupby(["month", "Category"], dropna=False)["ABS Amount"]
    .sum()
    .unstack(fill_value=0)
    .sort_index()
)

# remove any categories that have less than $100 total expenses in a month
monthly_category = monthly_category.loc[
    :, monthly_category.sum() >= 100
] 


In [24]:
import plotly.express as px
from IPython.display import HTML

# Build the same dataset used for the Matplotlib stacked bar chart
plotly_data = (
    monthly_category.reset_index()
    .melt(id_vars='month', var_name='Category', value_name='ABS Amount')
)
plotly_data['month'] = plotly_data['month'].astype(str)

fig = px.bar(
    plotly_data,
    x='month',
    y='ABS Amount',
    color='Category',
    labels={'month': 'Month'},
    title='Monthly Expenses by Category (Interactive)',
)
fig.update_layout(barmode='stack', height=800, width=1200, legend_title='Category')

HTML(fig.to_html(include_plotlyjs='cdn', full_html=False))


In [25]:

import plotly.graph_objects as go
from IPython.display import HTML

TREEMAP_QUARTER_START = pd.Period('2025-01', freq='M')

treemap_df = expenses_df.assign(MonthPeriod=expenses_df["Posted Date"].dt.to_period("M"))
month_periods = sorted(treemap_df["MonthPeriod"].unique())
if not month_periods:
    raise ValueError("No expenses available to visualize.")

period_labels = {
    period: period.to_timestamp().strftime("%b %Y")
    for period in month_periods
}

selection_options = [
    {"label": "Entire Dataset", "start": None, "end": None}
]

for period in month_periods:
    selection_options.append(
        {
            "label": f"Month – {period_labels[period]}",
            "start": period,
            "end": period,
        }
    )

quarter_ranges = {}
for period in month_periods:
    if period < TREEMAP_QUARTER_START:
        continue
    quarter_start_month = ((period.month - 1) // 3) * 3 + 1
    start_period = pd.Period(year=period.year, month=quarter_start_month, freq='M')
    if start_period < TREEMAP_QUARTER_START:
        start_period = TREEMAP_QUARTER_START
    end_period = start_period + 2
    if end_period > month_periods[-1]:
        end_period = month_periods[-1]
    quarter_ranges[start_period] = (start_period, end_period)

for start_period in sorted(quarter_ranges.keys()):
    start_period, end_period = quarter_ranges[start_period]
    quarter_number = ((start_period.month - 1) // 3) + 1
    start_label = start_period.to_timestamp().strftime("%b %Y")
    end_label = end_period.to_timestamp().strftime("%b %Y")
    if start_label == end_label:
        date_range_label = start_label
    else:
        date_range_label = f"{start_label} - {end_label}"
    selection_options.append(
        {
            "label": f"Quarter – Q{quarter_number} {start_period.year} ({date_range_label})",
            "start": start_period,
            "end": end_period,
        }
    )


def _filter_df(start_period, end_period):
    if start_period is None or end_period is None:
        return treemap_df
    mask = (treemap_df["MonthPeriod"] >= start_period) & (
        treemap_df["MonthPeriod"] <= end_period
    )
    return treemap_df.loc[mask]


fig = go.Figure()
buttons = []

for idx, option in enumerate(selection_options):
    filtered = _filter_df(option["start"], option["end"])
    total_by_category = (
        filtered.groupby("Category", dropna=False)["ABS Amount"].sum().reset_index()
    )
    if total_by_category["ABS Amount"].sum() == 0:
        trace = go.Treemap(
            labels=["No expenses in this selection"],
            parents=[""],
            values=[1],
            textinfo="label",
            hovertemplate="<b>No expenses in this selection</b><extra></extra>",
            visible=(idx == 0),
        )
    else:
        trace = go.Treemap(
            labels=total_by_category["Category"],
            parents=[""] * len(total_by_category),
            values=total_by_category["ABS Amount"],
            textinfo="label+value+percent entry",
            hovertemplate="<b>%{label}</b><br>$%{value:,.2f}<extra></extra>",
            visible=(idx == 0),
        )
    fig.add_trace(trace)
    buttons.append(
        dict(
            label=option["label"],
            method="update",
            args=[
                {"visible": [k == idx for k in range(len(selection_options))]},
                {"title": f"Total Expenses by Category Treemap ({option['label']})"},
            ],
        )
    )

fig.update_layout(
    height=800,
    width=1200,
    margin=dict(t=90, l=0, r=0, b=0),
    title="Total Expenses by Category Treemap (Entire Dataset)",
    updatemenus=[
        dict(
            type="dropdown",
            buttons=buttons,
            direction="down",
            x=0,
            y=1.18,
            xanchor="left",
            yanchor="top",
            pad=dict(r=10, t=10),
            showactive=True,
        )
    ],
)

fig.add_annotation(
    text="Select Entire Dataset, a Month, or a Quarter:",
    x=0,
    xanchor="left",
    y=1.25,
    yanchor="top",
    showarrow=False,
    xref="paper",
    yref="paper",
)

HTML(fig.to_html(include_plotlyjs="cdn", full_html=False))
